# 2.5 Automatic Differentiation

Calculating derivatives is the crucial step
in all the optimization algorithms
that we will use to train deep networks.
While the calculations are straightforward,
working them out by hand can be tedious and error-prone, 
and these issues only grow
as our models become more complex.

Fortunately all modern deep learning frameworks
take this work off our plates
by offering *automatic differentiation*
(often shortened to *autograd*). 
As we pass data through each successive function,
the framework builds a *computational graph* 
that tracks how each value depends on others.
To calculate derivatives, 
automatic differentiation 
works backwards through this graph
applying the chain rule. 
The computational algorithm for applying the chain rule
in this fashion is called *backpropagation*.

While autograd libraries have become
a hot concern over the past decade,
they have a long history. 
In fact the earliest references to autograd
date back over half of a century.
The core ideas behind modern backpropagation
date to a PhD thesis from 1980 
and were further developed in the late 1980s.
While backpropagation has become the default method 
for computing gradients, it is not the only option. 
For instance, the Julia programming language employs 
forward propagation. 
Before exploring methods, 
let's first master the autograd package.


In [1]:
import torch

## 2.5.1 A Simple Function

Let's assume that we are interested
in **differentiating the function
$y = 2\mathbf{x}^{\top}\mathbf{x}$
with respect to the column vector $\mathbf{x}$.**
To start, we assign `x` an initial value.


In [2]:
x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

**Before we calculate the gradient
of $y$ with respect to $\mathbf{x}$,
we need a place to store it.**
In general, we avoid allocating new memory
every time we take a derivative 
because deep learning requires 
successively computing derivatives
with respect to the same parameters
a great many times,
and we might risk running out of memory.
Note that the gradient of a scalar-valued function
with respect to a vector $\mathbf{x}$
is vector-valued with 
the same shape as $\mathbf{x}$.


In [3]:
# Can also create x = torch.arange(4.0, requires_grad=True)
x.requires_grad_(True)
x.grad  # The gradient is None by default

**We now calculate our function of `x` and assign the result to `y`.**


In [4]:
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

**We can now take the gradient of `y`
with respect to `x`** by calling 
its `backward` method.
Next, we can access the gradient 
via `x`'s `grad` attribute.


In [5]:
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

**We already know that the gradient of the function $y = 2\mathbf{x}^{\top}\mathbf{x}$
with respect to $\mathbf{x}$ should be $4\mathbf{x}$.**
We can now verify that the automatic gradient computation
and the expected result are identical.


In [6]:
x.grad == 4 * x

tensor([True, True, True, True])

**Now let's calculate 
another function of `x`
and take its gradient.**
Note that PyTorch does not automatically 
reset the gradient buffer 
when we record a new gradient. 
Instead, the new gradient
is added to the already-stored gradient.
This behavior comes in handy
when we want to optimize the sum 
of multiple objective functions.
To reset the gradient buffer,
we can call `x.grad.zero_()` as follows:


In [6]:
x.grad.zero_()  # Reset the gradient
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])